In [1]:
module Paris =
  struct
    type 'a pair = 'a * 'a
    let pair (a, b) = (a, b)
    let first (a, b) = a
    let second (a, b) = b
  end;;

  Paris.first;;
  let p = Paris.pair (1, 2);;
  Paris.first p;;
  List.map;;

module Paris :
  sig
    type 'a pair = 'a * 'a
    val pair : 'a * 'b -> 'a * 'b
    val first : 'a * 'b -> 'a
    val second : 'a * 'b -> 'b
  end


- : 'a * 'b -> 'a = <fun>


val p : int * int = (1, 2)


- : int = 1


- : ('a -> 'b) -> 'a list -> 'b list = <fun>


In [2]:
module Paris2 =
  struct
    type 'a pair = bool -> 'a
    let pair (a, b) = fun x -> if x then a else b
    let first ab = ab true
    let second ab = ab false
  end;;

open Paris2;;
pair;;
pair (4, 3) true;;

open List;;
map;;
fold_left;;

module Paris2 :
  sig
    type 'a pair = bool -> 'a
    val pair : 'a * 'a -> bool -> 'a
    val first : (bool -> 'a) -> 'a
    val second : (bool -> 'a) -> 'a
  end


- : 'a * 'a -> bool -> 'a = <fun>


- : int = 4


- : ('a -> 'b) -> 'a list -> 'b list = <fun>


- : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


We model sets of values using modules with signature

Furthermore, we define a signature for maps (mappings from keys to values):


In [ ]:
module type Set = sig
  type t
  val to_string : t -> string
end

module type Map = sig
  type key
  type value
  type t

  val empty : t
  val set : key -> value -> t -> t
  val get : key -> t -> value
  val get_opt : key -> t -> value option
  (* val clear : key -> t -> t *)
  val to_string : t -> string
end

Where the functions have the following semantics:
- set updates the mapping such that key is now mapped to value.
- get retrieves the value for the given key and throws a `Not_found` exception if no
such key exists in the map.
- get_opt retrieves the value for the given key or `None` if the key does not exist.
- to_string produces a string representation for the mapping, e.g.:
`"{ 1 -> \"x\", 5 -> \"y\" }"`

Perform these tasks:


Hint: If a module’s signature is too abstract to be used with concrete values, make use of sharing constraints.

1. Implement a module `StringSet` of signature `Set` to model sets of strings.

In [5]:
module type Set = sig
  type t
  val to_string : t -> string
end

module type Set = sig type t val to_string : t -> string end


In [6]:
module StringSet : Set with type t = string = struct
  type t = string
  let to_string s = "\"" ^ s ^ "\""
end

module StringSet : sig type t = string val to_string : t -> string end


In [8]:
StringSet.to_string;;
StringSet.to_string "this is a module";;

- : StringSet.t -> string = <fun>


- : string = "\"this is a module\""


In [8]:
StringSet.to_string "string";;

- : string = "\"string\""


$2$. Define a signature `OrderedSet` that extends the `Set` signature by a **compare** function with the usual type.

the keyword `include` allows to include the definitions of another
module into the present module ...

In [13]:
(* Review *)
module A = struct let x = 1 end;;

module B = struct
  open A (*but not use it, save the modul name*)
  let y = 2
end;;

module C = struct
  include A (* like C to include the head file*)
  include B
end;;

module A : sig val x : int end


module B : sig val y : int end


module C : sig val x : int val y : int end


- : 'a -> 'a -> int = <fun>


- : int = -1


In [20]:
module type Set = sig
  type t
  val to_string : t -> string
end;;

module type OrderedSet = sig
   include Set
   val compare : t -> t -> int
end;;

module IntSet : OrderedSet with type t = int = struct
  type t = int
  let to_string i = string_of_int i
  let compare first second = Pervasives.compare first second
end;;

Pervasives.compare 1 2;;
Pervasives.compare 2 1;;
Pervasives.compare 1 1;;
IntSet.compare;;
IntSet.to_string 8;;

module type Set = sig type t val to_string : t -> string end


module type OrderedSet =
  sig type t val to_string : t -> string val compare : t -> t -> int end


module IntSet :
  sig
    type t = int
    val to_string : t -> string
    val compare : t -> t -> int
  end


- : int = -1


- : int = 1


- : int = 0


- : IntSet.t -> IntSet.t -> int = <fun>


- : string = "8"


In [18]:
module type OrderedSet = sig
  include Set
  val compare : t -> t -> int (* first > second then positive, otherwise negative*)
end

module IntSet : OrderedSet with type t = int = struct
  type t = int
  let compare = compare  (*val compare : 'a -> 'a -> int, from Module Pervasives*)
  let to_string = string_of_int
end

module type OrderedSet =
  sig type t val to_string : t -> string val compare : t -> t -> int end


module IntSet :
  sig
    type t = int
    val to_string : t -> string
    val compare : t -> t -> int
  end


In [24]:
IntSet.compare 1 2;;
IntSet.compare 2 0;;
IntSet.to_string 1;;
open List;;
map;;
fold_left;;
open IntSet;;
to_string;;

- : int = -1


- : int = 1


- : string = "1"


- : ('a -> 'b) -> 'a list -> 'b list = <fun>


- : ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a = <fun>


- : IntSet.t -> string = <fun>


$3$. Implement a functor `BTreeMap` that realizes the `Map` signature and uses a binary
tree to store key-value-pairs. The functor takes key and value sets as arguments.
(In my group eariler we had a version without module and functor)

<br>
Since (almost) everything in Ocaml is higher order, it is no surprise that
there are modules of higher order: Functors.
- A functor receives a sequence of modules as parameters.
- The functor’s body is a module where the functor’s parameters can
be used.
- The result is a new module, which is defined relative to the
modules passed as parameters.

In [26]:
module type Set = sig
  type t
  val to_string : t -> string
end

module type OrderedSet = sig
  include Set
  val compare : t -> t -> int (* compare enable the map and btree*)
end

module type Map = sig
  type key
  type value
  type t

  val empty : t
  val set : key -> value -> t -> t
  val get : key -> t -> value
  val get_opt : key -> t -> value option
  val to_string : t -> string
end

(* OrderedSet, Set are sig*)

(* Basic Structure *)
module BTreeMap (K : OrderedSet) (V : Set) : Map
          with type key = K.t and type value = V.t = struct
  type key = K.t
  type value = V.t
  type t = Empty | Node of (key * value * t * t)

  let empty = Empty
  
  (* impl set get get_opt *)
  (* try to reuse K.compare to compare diff. keys *)
  
  let rec set k v t = 
    match t with Empty -> Node(k, v, Empty, Empty)
      | Node (k', v', l, r) -> 
        let com = K.compare k k'
        in
        if com > 0 then Node(k', v', l, set k v r)
        else if com < 0 then Node(k', v', set k v l, r)
        else Node(k, v, l, r)
    
  let rec get_opt k t = 
    match t with Empty -> None
      | Node (k', v', l, r) -> 
        let com = K.compare k k'
        in 
        if com > 0 then get_opt k r
        else if com < 0 then get_opt k l
        else Some v'
        
  let get k t = 
    match get_opt k t with None -> raise Not_found
      | Some v -> v
  
(* tree to a node list - keep ordered relation*)
  let rec to_list = function Empty -> []
    | Node (k, v, l, r) -> to_list l @ (k,v)::to_list r

  let to_string m =
    List.map (fun (k,v) -> K.to_string k ^ " -> " ^ V.to_string v) (to_list m)
    |> String.concat ", "
    |> Printf.sprintf "{ %s }"
end

module type Set = sig type t val to_string : t -> string end


module type OrderedSet =
  sig type t val to_string : t -> string val compare : t -> t -> int end


module type Map =
  sig
    type key
    type value
    type t
    val empty : t
    val set : key -> value -> t -> t
    val get : key -> t -> value
    val get_opt : key -> t -> value option
    val to_string : t -> string
  end


module BTreeMap :
  functor (K : OrderedSet) (V : Set) ->
    sig
      type key = K.t
      type value = V.t
      type t
      val empty : t
      val set : key -> value -> t -> t
      val get : key -> t -> value
      val get_opt : key -> t -> value option
      val to_string : t -> string
    end


In [19]:
(* OrderedSet, Set are sig*)

(* Basic Structure *)
module BTreeMap (K : OrderedSet) (V : Set) : Map
          with type key = K.t and type value = V.t = struct
  type key = K.t
  type value = V.t
  type t = Empty | Node of key * value * t * t

  let empty = Empty
  
  (* more things here*)
  let rec set k v t = 
      match t with Empty -> Node(k, v, Empty, Empty)
    | Node(k', v', l, r) ->
      let comp = K.compare k k'
      in
      if comp < 0 then Node(k', v', set k v l, r)
      else if comp > 0 then Node(k', v', l, set k v r)
      else Node(k', v, l, r)
  
(* tree to a node list - keep ordered relation*)
  let rec to_list = function Empty -> []
    | Node (k, v, l, r) -> to_list l @ (k,v)::to_list r

  let to_string m =
    List.map (fun (k,v) -> K.to_string k ^ " -> " ^ V.to_string v) (to_list m)
    |> String.concat ", "
    |> Printf.sprintf "{ %s }"
end

error: compile_error

In [58]:
module BTreeMap (K : OrderedSet) (V : Set) : Map
          with type key = K.t and type value = V.t = struct
  type key = K.t
  type value = V.t
  type t = Empty | Node of key * value * t * t

  let empty = Empty

  let rec set k v t = 
   match t with Empty -> Node (k, v, Empty, Empty)
     | Node (k', v', l, r) ->
      let diff = K.compare k k' in
      if diff < 0 then Node (k', v', set k v l, r)
      else if diff > 0 then Node (k', v', l, set k v r)
      else Node (k', v, l, r) (*reset the value*)

  let rec get_opt k t = 
   match t with Empty -> None
    | Node (k', v, l, r) ->
      let diff = K.compare k k' in
      if diff < 0 then get_opt k l
      else if diff > 0 then get_opt k r
      else Some v

  let get k m = match get_opt k m with Some v -> v
    | None -> raise Not_found

  (* tree to a node list - keep ordered relation*)
  let rec to_list = function Empty -> []
    | Node (k, v, l, r) -> to_list l @ (k,v)::to_list r

  let to_string m =
    List.map (fun (k,v) -> K.to_string k ^ " -> " ^ V.to_string v) (to_list m)
    |> String.concat ", "
    |> Printf.sprintf "{ %s }"
end

module BTreeMap :
  functor (K : OrderedSet) (V : Set) ->
    sig
      type key = K.t
      type value = V.t
      type t
      val empty : t
      val set : key -> value -> t -> t
      val get : key -> t -> value
      val get_opt : key -> t -> value option
      val to_string : t -> string
    end


$4$. Implement a `StringSet` and an `ordered IntSet` module.


In [30]:
(* implement the sig *)

module StringSet : Set with type t = string = struct
  type t = string
  let to_string s = "\"" ^ s ^ "\""
end

module IntSet : OrderedSet with type t = int = struct
  type t = int
  let compare = compare
  let to_string = string_of_int
end

module StringSet : sig type t = string val to_string : t -> string end


module IntSet :
  sig
    type t = int
    val to_string : t -> string
    val compare : t -> t -> int
  end


$5$. Use the `BTreeMap` functor to define modules for **int-to-string** and **int-to-int maps**.

In [33]:
module IntStringMap = BTreeMap(IntSet)(StringSet)
module IntIntMap = BTreeMap(IntSet)(IntSet)

(* Now we have something to play with *)

module IntStringMap :
  sig
    type key = IntSet.t
    type value = StringSet.t
    type t = BTreeMap(IntSet)(StringSet).t
    val empty : t
    val set : key -> value -> t -> t
    val get : key -> t -> value
    val get_opt : key -> t -> value option
    val to_string : t -> string
  end


module IntIntMap :
  sig
    type key = IntSet.t
    type value = IntSet.t
    type t = BTreeMap(IntSet)(IntSet).t
    val empty : t
    val set : key -> value -> t -> t
    val get : key -> t -> value
    val get_opt : key -> t -> value option
    val to_string : t -> string
  end


In [31]:
module PAMap = BTreeMap(IntSet);;

module PAMap :
  functor (V : Set) ->
    sig
      type key = IntSet.t
      type value = V.t
      type t = BTreeMap(IntSet)(V).t
      val empty : t
      val set : key -> value -> t -> t
      val get : key -> t -> value
      val get_opt : key -> t -> value option
      val to_string : t -> string
    end


In [34]:
let t = IntIntMap.empty;;
print_endline(IntIntMap.to_string t);;
let t = IntIntMap.set 1 1 t;;
let t = IntIntMap.set 2 4 t;;
let t = IntIntMap.set 3 9 t;;
let t = IntIntMap.set 4 16 t;;
print_endline(IntIntMap.to_string t);;

val t : IntIntMap.t = <abstr>


{  }


- : unit = ()


val t : IntIntMap.t = <abstr>


val t : IntIntMap.t = <abstr>


val t : IntIntMap.t = <abstr>


val t : IntIntMap.t = <abstr>


{ 1 -> 1, 2 -> 4, 3 -> 9, 4 -> 16 }


- : unit = ()


In [54]:
(* or use a List.fold_left*)
let ii_tree = List.fold_left (fun tree key -> IntIntMap.set key (key * key) tree) IntIntMap.empty [1;2;3;4];;
print_endline(IntIntMap.to_string ii_tree);;

val ii_tree : IntIntMap.t = <abstr>


{ 1 -> 1, 2 -> 4, 3 -> 9, 4 -> 16 }


- : unit = ()


$6$. Insert some values into an **int-to-int** map and **int-to-string** map and print its string representation.


In [35]:

let values = [2;4;6;7;8;31]

let ii = List.fold_left (fun m k -> IntIntMap.set k (k * k) m) IntIntMap.empty values
let _ = print_endline (IntIntMap.to_string ii)

let m = List.fold_left (fun m k -> IntStringMap.set k (string_of_int (k * k)) m) IntStringMap.empty values
let _ = print_endline (IntStringMap.to_string m)

val values : int list = [2; 4; 6; 7; 8; 31]


val ii : IntIntMap.t = <abstr>


{ 2 -> 4, 4 -> 16, 6 -> 36, 7 -> 49, 8 -> 64, 31 -> 961 }


- : unit = ()


val m : IntStringMap.t = <abstr>


{ 2 -> "4", 4 -> "16", 6 -> "36", 7 -> "49", 8 -> "64", 31 -> "961" }


- : unit = ()
